In [6]:
pip install fuzzywuzzy

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install python-Levenshtein

Processing /Users/liuyuxuan/Library/Caches/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342/python_Levenshtein-0.12.0-cp37-cp37m-macosx_10_7_x86_64.whl
Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process 
import heapq
import math

In [3]:
car_data = pd.read_csv('car_data_kaggle.csv')
insurance_data = pd.read_csv('insurance.csv')

In [4]:
# field of interest
# Make, Model, Year, highway MPG, city mpg, MSRP
car_data

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11909,Acura,ZDX,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,46120
11910,Acura,ZDX,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,56670
11911,Acura,ZDX,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,50620
11912,Acura,ZDX,2013,premium unleaded (recommended),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,50920


In [5]:
insurance_data

,Make,Model,Price:AK,Price:AL,Price:AZ,Price:AR,Price:CA,Price:CO,Price:CT,Price:DE,...,Price:SD,Price:TN,Price:TX,Price:UT,Price:VT,Price:VA,Price:WA,Price:WV,Price:WI,Price:WY
0,Acura,ILX 2.4L,1407.0,1286.0,1543.0,1757.0,2124.0,1926.0,1873.0,2119.0,...,1607.0,1494.0,1420.0,2048.0,1206.0,1331.0,1237.0,1582.0,1625.0,1055.0
1,Acura,ILX 2.4L Prem A-SPEC,1430.0,1318.0,1568.0,1792.0,2146.0,1962.0,1903.0,2160.0,...,1655.0,1550.0,1437.0,2123.0,1290.0,1356.0,1260.0,1621.0,1669.0,1076.0
2,Acura,ILX 2.4L Premium,1404.0,1318.0,1534.0,1745.0,2134.0,1913.0,1867.0,2119.0,...,1644.0,1519.0,1404.0,2108.0,1232.0,1328.0,1238.0,1609.0,1604.0,1056.0
3,Acura,ILX 2.4L Special Edition,1448.0,1287.0,1619.0,1831.0,2124.0,1989.0,1928.0,2181.0,...,1611.0,1571.0,1469.0,2056.0,1247.0,1372.0,1272.0,1586.0,1724.0,1088.0
4,Acura,ILX 2.4L Tech,1412.0,1327.0,1540.0,1753.0,2134.0,1919.0,1874.0,2131.0,...,1651.0,1526.0,1410.0,2117.0,1249.0,1335.0,1246.0,1616.0,1609.0,1060.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1458,Volvo,XC90 T8,2487.0,1706.0,2560.0,2126.0,2515.0,3847.0,3158.0,2487.0,...,1845.0,2227.0,1616.0,2459.0,1634.0,2250.0,1351.0,2082.0,2800.0,1236.0
1459,Volvo,XC90 T8 Excellence,2095.0,1832.0,2210.0,2400.0,3007.0,3540.0,2402.0,2695.0,...,2300.0,2707.0,1822.0,3212.0,1879.0,1793.0,1559.0,2165.0,2483.0,1515.0
1460,Volvo,XC90 T8 Inscription,1770.5,1537.5,2187.0,2205.0,2513.0,3414.5,2765.5,2453.5,...,1915.0,2249.0,1663.5,2439.5,1683.0,1864.0,1356.0,1852.0,2139.5,1334.5
1461,Volvo,XC90 T8 Momentum,1684.5,1492.0,2146.0,2005.0,2455.0,3280.5,2550.5,2206.5,...,1841.0,1987.5,1530.5,2508.0,1584.5,1742.5,1311.0,1818.0,1914.0,1216.5


In [6]:
# make_set = set(insurance_data['Make'])

make_model_str_set = set()

for row in insurance_data[['Make', 'Model']].iterrows():
#     print(row[1]['Make'])
#     print(row[1]['Model'])
    make_model_str_set.add(" ".join([row[1]['Make'], row[1]['Model']]))
print(make_model_str_set)

{'Mazda Mazda3 Sport', 'Mercedes CLA45 AMG 4Matic', 'Porsche Panamera 4 Sport Turismo', 'Toyota Avalon XLE Touring', 'Toyota Prius C One', 'Chevrolet Malibu LT', 'Volkswagen Atlas Execline+ R-Line', 'Mercedes GLE43 AMG', 'Hyundai Veloster Ultimate', 'Lexus RC 350 F-Sport', 'Audi R8 5.2L V10 Quattro Plus', 'Genesis G90 Ultimate', 'GMC Acadia Terrain-2', 'Alfa-Romeo Giulia TI Q4', 'Ford Focus SEL', 'Porsche 911 Carrera 2', 'Porsche Panamera 4S Sport Turismo', 'Audi A8 L 3.0T Quattro', 'Ford Mustang', 'Mini Cooper Clubman S', 'Subaru Crosstrek Premium', 'Ford Explorer', 'Jaguar XE SV Project 8', 'Dodge Journey SE', 'Mercedes GLA45 4MATIC', 'Land Rover Range Rover Evoq HSE', 'Subaru Legacy 2.5I Premium', 'Ford Fiesta ST Line', 'Acura RDX', 'Audi Etron Premium Ultra', 'Cadillac CT6 Sport', 'Honda Civic Sport Touring', 'Mazda CX-3 Grand Touring', 'Subaru Forester 2.5I Limited', 'BMW I3 S', 'Nissan GT-R', 'Chevrolet Colorado Work Truck', 'GMC Terrain SLE', 'Mazda Mazda3 Touring', 'Nissan Fron

In [7]:
fuzz.ratio("330i XDrive", "XDrive40i")

60

In [8]:
fuzz.ratio("330i XDrive", "XDrive330i")

57

In [9]:
cars_dot_com = pd.read_csv('sample_cars_dot_com.csv')

In [10]:
cars_dot_com = cars_dot_com.set_index('name')
cars_dot_com

,Unnamed: 0,price
name,,
2019 Cadillac XT4 Sport,0,"$44,430"
2019 Mazda CX-5 Sport,1,"$25,814"
2020 GMC Sierra 1500 Elevation,2,"$45,825"
2020 Toyota RAV4 LE,3,"$26,441"
2020 Chevrolet Traverse LT Cloth,4,"$33,851"
2019 Ford Ranger XLT,5,"$37,903"
2020 Nissan Maxima 3.5 SL,6,"$38,597"
2020 GMC Acadia SLE,7,"$34,024"
2020 GMC Terrain SLT,8,"$31,496"


In [11]:
# manual_df = pd.DataFrame(columns = ['First', "Second", "Third"])
# for name in cars_dot_com.index:
# #     print(name)
# #     word_list = name.split(" ")
# #     print(word_list)
# #     year = int(word_list[0])
# #     print(year)
# #     make = ""
# #     model = ""
# #     trim = ""
#     make_model_name = name[5:len(name)]
# #     best_score = -1
# #     best_match = ""
#     h = []
#     for known_name in make_model_str_set:
#         score = fuzz.ratio(make_model_name, known_name)
#         if score == 100:
#             h = [(score, (name, known_name))]
#             break
#         else:
#             heapq.heappush(h, (score, (name, known_name)))
#     while (len(h)) > 3:
#         heapq.heappop(h)
# #     print(h)
#     if (len(h)) > 1:
#         manual_df = manual_df.append({'First': heapq.heappop(h),'Second': heapq.heappop(h),'Third': heapq.heappop(h)} , ignore_index=True)
# manual_df.to_csv("manual_checklist.csv")
        
# #         if score > best_score:
# #             best_score = score
# #             best_match = known_name
# #     print("Matched: " + str((name, best_match)) + " by score " + str(best_score))

In [12]:
decision_df = pd.read_csv("manual_checklist.csv")
decision_df

,Unnamed: 0,First,Second,Third
0,0,"(85, ('2020 Chevrolet Traverse LT Cloth', 'Che...","(88, ('2020 Chevrolet Traverse LT Cloth', 'Che...","(89, ('2020 Chevrolet Traverse LT Cloth', 'Che..."
1,1,"(64, ('2019 Ford Ranger XLT', 'Ford Edge SEL'))","(67, ('2019 Ford Ranger XLT', 'Ford Mustang GT'))","(69, ('2019 Ford Ranger XLT', 'Ford Explorer X..."
2,2,"(85, ('2020 Nissan Maxima 3.5 SL', 'Nissan Alt...","(86, ('2020 Nissan Maxima 3.5 SL', 'Nissan Max...","(89, ('2020 Nissan Maxima 3.5 SL', 'Nissan Max..."
3,3,"(93, ('2020 GMC Acadia SLE', 'GMC Acadia SLE-1'))","(93, ('2020 GMC Acadia SLE', 'GMC Acadia SLE-2'))","(96, ('2020 GMC Acadia SLE', 'GMC Acadia SL'))"
4,4,"(82, ('2020 Mazda Mazda6 Grand Touring', 'Mazd...","(83, ('2020 Mazda Mazda6 Grand Touring', 'Mazd...","(96, ('2020 Mazda Mazda6 Grand Touring', 'Mazd..."


In [ ]:
special_mapping = dict()
for row in decision_df.iterrows():
    if (row[1]['Choice'] != 0):
        print(list(row[1][['First', 'Second', 'Third']])[row[1]['Choice'] - 1])

In [17]:
cars_cleaned_df = pd.read_csv('cars_cleaned_grouped.csv')
cars_cleaned_df

,name,price,city_mpg,highway_mpg
0,2017 Kia Forte S,15989.0,29.0,38.0
1,2017 Mercedes-Benz E300,57175.0,22.0,30.0
2,2018 BMW 340 i xDrive,51444.0,21.0,31.0
3,2018 BMW 740 i xDrive,98395.0,20.0,29.0
4,2018 BMW X2 sDrive28i,35995.0,23.0,32.0
...,...,...,...,...
840,2020 Volvo XC40 T5 Momentum,43280.0,22.0,30.0
841,2020 Volvo XC60 T5 Momentum,49970.0,20.0,27.0
842,2020 Volvo XC60 T5 R-Design,53755.0,20.0,27.0
843,2020 Volvo XC90 T5 Momentum 7 Passenger,52005.0,18.0,26.0


In [267]:
name_map = dict()
manual_df = pd.DataFrame(columns = ['First', "Second", "Third"])
good_match_count = 0
for name in cars_cleaned_df['name']:
#     print(name)
#     word_list = name.split(" ")
#     print(word_list)
#     year = int(word_list[0])
#     print(year)
#     make = ""
#     model = ""
#     trim = ""
    make_model_name = name[5:len(name)]
#     best_score = -1
#     best_match = ""
    h = []
    for known_name in make_model_str_set:
        score = fuzz.ratio(make_model_name, known_name)
        if score == 100:
            h = [(score, (name, known_name))]
            name_map[name] = known_name
            good_match_count += 1
#             print(h)
            break
        else:
            heapq.heappush(h, (score, (name, known_name)))
    while (len(h)) > 3:
        heapq.heappop(h)
#     print(h)
    if (len(h)) > 1:
        manual_df = manual_df.append({'First': heapq.heappop(h),'Second': heapq.heappop(h),'Third': heapq.heappop(h)} , ignore_index=True)
manual_df.to_csv("manual_checklist.csv")

print(len(name_map))
        
#         if score > best_score:
#             best_score = score
#             best_match = known_name
#     print("Matched: " + str((name, best_match)) + " by score " + str(best_score))

365


In [268]:
good_match_count

365

In [269]:
name_map

{'2017 Kia Forte S': 'Kia Forte S',
 '2018 Buick Envision Premium II': 'Buick Envision Premium II',
 '2018 Chevrolet Trax LT': 'Chevrolet Trax LT',
 '2018 Chrysler Pacifica Limited': 'Chrysler Pacifica Limited',
 '2018 Ford Explorer XLT': 'Ford Explorer XLT',
 '2018 Ford Fusion SE': 'Ford Fusion SE',
 '2018 Honda Accord EX-L': 'Honda Accord EX-L',
 '2018 Jeep Cherokee Overland': 'Jeep Cherokee Overland',
 '2018 Jeep Wrangler Unlimited Sahara': 'Jeep Wrangler Unlimited Sahara',
 '2018 Kia Niro Touring': 'Kia Niro Touring',
 '2018 Toyota Prius Four': 'Toyota Prius Four',
 '2019 Audi A4 2.0T Premium Plus': 'Audi A4 2.0T Premium Plus',
 '2019 Audi Q5 2.0T Prestige': 'Audi Q5 2.0T Prestige',
 '2019 Buick Enclave Essence': 'Buick Enclave Essence',
 '2019 Buick Enclave Premium': 'Buick Enclave Premium',
 '2019 Buick Encore Preferred': 'Buick Encore Preferred',
 '2019 Buick Encore Sport Touring': 'Buick Encore Sport Touring',
 '2019 Buick Envision Essence': 'Buick Envision Essence',
 '2019 Bui

In [270]:
manual_decision = pd.read_csv('Manual Checklist - Sheet1.csv')


In [271]:
for row in manual_decision.iterrows():
    if not math.isnan(row[1]['Choice']) and row[1]['Choice'] != 0 and row[1]['Choice'] in [0,1,2,3]:
        decision_str = list(row[1][['First', 'Second', 'Third']])[int(row[1]['Choice']) - 1]
        decision = decision_str.split(',')
        decision[1] = decision[1].replace('(','').replace(')','').replace("""'""",'').strip(' ')
        decision[2] = decision[2].replace('(','').replace(')','').replace("""'""",'').strip(' ')
        assert decision[1] not in name_map.keys(), decision[1]
        name_map[decision[1]] = decision[2]

print(len(name_map))

660


In [272]:
# # Fuzzy match with a fixed score
# name_map = dict()
# manual_df = pd.DataFrame(columns = ['First', "Second", "Third"])
# good_match_count = 0
# for name in cars_cleaned_df['name']:

#     make_model_name = name[5:len(name)]
#     best_score = -1
#     best_match = ""
#     h = []
#     for known_name in make_model_str_set:
#         score = fuzz.ratio(make_model_name, known_name)
#         if score > best_score:
#             best_score = score
#             best_match = known_name
#     if best_score >= 85:
#         name_map[name] = best_match
#     print("Matched: " + str((name, best_match)) + " by score " + str(best_score))
# print(len(name_map))
        


In [273]:
# name from cars.com, assembled name from insurance
name_map

{'2017 Kia Forte S': 'Kia Forte S',
 '2018 Buick Envision Premium II': 'Buick Envision Premium II',
 '2018 Chevrolet Trax LT': 'Chevrolet Trax LT',
 '2018 Chrysler Pacifica Limited': 'Chrysler Pacifica Limited',
 '2018 Ford Explorer XLT': 'Ford Explorer XLT',
 '2018 Ford Fusion SE': 'Ford Fusion SE',
 '2018 Honda Accord EX-L': 'Honda Accord EX-L',
 '2018 Jeep Cherokee Overland': 'Jeep Cherokee Overland',
 '2018 Jeep Wrangler Unlimited Sahara': 'Jeep Wrangler Unlimited Sahara',
 '2018 Kia Niro Touring': 'Kia Niro Touring',
 '2018 Toyota Prius Four': 'Toyota Prius Four',
 '2019 Audi A4 2.0T Premium Plus': 'Audi A4 2.0T Premium Plus',
 '2019 Audi Q5 2.0T Prestige': 'Audi Q5 2.0T Prestige',
 '2019 Buick Enclave Essence': 'Buick Enclave Essence',
 '2019 Buick Enclave Premium': 'Buick Enclave Premium',
 '2019 Buick Encore Preferred': 'Buick Encore Preferred',
 '2019 Buick Encore Sport Touring': 'Buick Encore Sport Touring',
 '2019 Buick Envision Essence': 'Buick Envision Essence',
 '2019 Bui

In [154]:
insurance_data

,Make,Model,Price:AK,Price:AL,Price:AZ,Price:AR,Price:CA,Price:CO,Price:CT,Price:DE,...,Price:SD,Price:TN,Price:TX,Price:UT,Price:VT,Price:VA,Price:WA,Price:WV,Price:WI,Price:WY
0,Acura,ILX 2.4L,1407.0,1286.0,1543.0,1757.0,2124.0,1926.0,1873.0,2119.0,...,1607.0,1494.0,1420.0,2048.0,1206.0,1331.0,1237.0,1582.0,1625.0,1055.0
1,Acura,ILX 2.4L Prem A-SPEC,1430.0,1318.0,1568.0,1792.0,2146.0,1962.0,1903.0,2160.0,...,1655.0,1550.0,1437.0,2123.0,1290.0,1356.0,1260.0,1621.0,1669.0,1076.0
2,Acura,ILX 2.4L Premium,1404.0,1318.0,1534.0,1745.0,2134.0,1913.0,1867.0,2119.0,...,1644.0,1519.0,1404.0,2108.0,1232.0,1328.0,1238.0,1609.0,1604.0,1056.0
3,Acura,ILX 2.4L Special Edition,1448.0,1287.0,1619.0,1831.0,2124.0,1989.0,1928.0,2181.0,...,1611.0,1571.0,1469.0,2056.0,1247.0,1372.0,1272.0,1586.0,1724.0,1088.0
4,Acura,ILX 2.4L Tech,1412.0,1327.0,1540.0,1753.0,2134.0,1919.0,1874.0,2131.0,...,1651.0,1526.0,1410.0,2117.0,1249.0,1335.0,1246.0,1616.0,1609.0,1060.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1458,Volvo,XC90 T8,2487.0,1706.0,2560.0,2126.0,2515.0,3847.0,3158.0,2487.0,...,1845.0,2227.0,1616.0,2459.0,1634.0,2250.0,1351.0,2082.0,2800.0,1236.0
1459,Volvo,XC90 T8 Excellence,2095.0,1832.0,2210.0,2400.0,3007.0,3540.0,2402.0,2695.0,...,2300.0,2707.0,1822.0,3212.0,1879.0,1793.0,1559.0,2165.0,2483.0,1515.0
1460,Volvo,XC90 T8 Inscription,1770.5,1537.5,2187.0,2205.0,2513.0,3414.5,2765.5,2453.5,...,1915.0,2249.0,1663.5,2439.5,1683.0,1864.0,1356.0,1852.0,2139.5,1334.5
1461,Volvo,XC90 T8 Momentum,1684.5,1492.0,2146.0,2005.0,2455.0,3280.5,2550.5,2206.5,...,1841.0,1987.5,1530.5,2508.0,1584.5,1742.5,1311.0,1818.0,1914.0,1216.5


In [274]:
make_model_str_set = set()
for row in insurance_data[['Make', 'Model']].iterrows():
    make_model_str_set.add(" ".join([row[1]['Make'], row[1]['Model']]))
print(make_model_str_set)

{'Mazda Mazda3 Sport', 'Mercedes CLA45 AMG 4Matic', 'Porsche Panamera 4 Sport Turismo', 'Toyota Avalon XLE Touring', 'Toyota Prius C One', 'Chevrolet Malibu LT', 'Volkswagen Atlas Execline+ R-Line', 'Mercedes GLE43 AMG', 'Hyundai Veloster Ultimate', 'Lexus RC 350 F-Sport', 'Audi R8 5.2L V10 Quattro Plus', 'Genesis G90 Ultimate', 'GMC Acadia Terrain-2', 'Alfa-Romeo Giulia TI Q4', 'Ford Focus SEL', 'Porsche 911 Carrera 2', 'Porsche Panamera 4S Sport Turismo', 'Audi A8 L 3.0T Quattro', 'Ford Mustang', 'Mini Cooper Clubman S', 'Subaru Crosstrek Premium', 'Ford Explorer', 'Jaguar XE SV Project 8', 'Dodge Journey SE', 'Mercedes GLA45 4MATIC', 'Land Rover Range Rover Evoq HSE', 'Subaru Legacy 2.5I Premium', 'Ford Fiesta ST Line', 'Acura RDX', 'Audi Etron Premium Ultra', 'Cadillac CT6 Sport', 'Honda Civic Sport Touring', 'Mazda CX-3 Grand Touring', 'Subaru Forester 2.5I Limited', 'BMW I3 S', 'Nissan GT-R', 'Chevrolet Colorado Work Truck', 'GMC Terrain SLE', 'Mazda Mazda3 Touring', 'Nissan Fron

In [275]:
insurance_data = pd.read_csv('insurance.csv')
assembled_name = list()
for row in insurance_data.iterrows():
    assembled_name.append(" ".join([row[1]['Make'], row[1]['Model']]))
insurance_data.insert(loc=len(insurance_data.columns),column='assembled_name', value=assembled_name)

In [276]:
insurance_data

,Make,Model,Price:AK,Price:AL,Price:AZ,Price:AR,Price:CA,Price:CO,Price:CT,Price:DE,...,Price:TN,Price:TX,Price:UT,Price:VT,Price:VA,Price:WA,Price:WV,Price:WI,Price:WY,assembled_name
0,Acura,ILX 2.4L,1407.0,1286.0,1543.0,1757.0,2124.0,1926.0,1873.0,2119.0,...,1494.0,1420.0,2048.0,1206.0,1331.0,1237.0,1582.0,1625.0,1055.0,Acura ILX 2.4L
1,Acura,ILX 2.4L Prem A-SPEC,1430.0,1318.0,1568.0,1792.0,2146.0,1962.0,1903.0,2160.0,...,1550.0,1437.0,2123.0,1290.0,1356.0,1260.0,1621.0,1669.0,1076.0,Acura ILX 2.4L Prem A-SPEC
2,Acura,ILX 2.4L Premium,1404.0,1318.0,1534.0,1745.0,2134.0,1913.0,1867.0,2119.0,...,1519.0,1404.0,2108.0,1232.0,1328.0,1238.0,1609.0,1604.0,1056.0,Acura ILX 2.4L Premium
3,Acura,ILX 2.4L Special Edition,1448.0,1287.0,1619.0,1831.0,2124.0,1989.0,1928.0,2181.0,...,1571.0,1469.0,2056.0,1247.0,1372.0,1272.0,1586.0,1724.0,1088.0,Acura ILX 2.4L Special Edition
4,Acura,ILX 2.4L Tech,1412.0,1327.0,1540.0,1753.0,2134.0,1919.0,1874.0,2131.0,...,1526.0,1410.0,2117.0,1249.0,1335.0,1246.0,1616.0,1609.0,1060.0,Acura ILX 2.4L Tech
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1458,Volvo,XC90 T8,2487.0,1706.0,2560.0,2126.0,2515.0,3847.0,3158.0,2487.0,...,2227.0,1616.0,2459.0,1634.0,2250.0,1351.0,2082.0,2800.0,1236.0,Volvo XC90 T8
1459,Volvo,XC90 T8 Excellence,2095.0,1832.0,2210.0,2400.0,3007.0,3540.0,2402.0,2695.0,...,2707.0,1822.0,3212.0,1879.0,1793.0,1559.0,2165.0,2483.0,1515.0,Volvo XC90 T8 Excellence
1460,Volvo,XC90 T8 Inscription,1770.5,1537.5,2187.0,2205.0,2513.0,3414.5,2765.5,2453.5,...,2249.0,1663.5,2439.5,1683.0,1864.0,1356.0,1852.0,2139.5,1334.5,Volvo XC90 T8 Inscription
1461,Volvo,XC90 T8 Momentum,1684.5,1492.0,2146.0,2005.0,2455.0,3280.5,2550.5,2206.5,...,1987.5,1530.5,2508.0,1584.5,1742.5,1311.0,1818.0,1914.0,1216.5,Volvo XC90 T8 Momentum


In [277]:
cars_cleaned_df = pd.read_csv('cars_cleaned_grouped.csv')
cars_cleaned_df

,name,price,city_mpg,highway_mpg
0,2017 Kia Forte S,15989.0,29.0,38.0
1,2017 Mercedes-Benz E300,57175.0,22.0,30.0
2,2018 BMW 340 i xDrive,51444.0,21.0,31.0
3,2018 BMW 740 i xDrive,98395.0,20.0,29.0
4,2018 BMW X2 sDrive28i,35995.0,23.0,32.0
...,...,...,...,...
840,2020 Volvo XC40 T5 Momentum,43280.0,22.0,30.0
841,2020 Volvo XC60 T5 Momentum,49970.0,20.0,27.0
842,2020 Volvo XC60 T5 R-Design,53755.0,20.0,27.0
843,2020 Volvo XC90 T5 Momentum 7 Passenger,52005.0,18.0,26.0


In [287]:
insurance_data['price'] = np.nan
insurance_data['city_mpg'] = np.nan
insurance_data['highway_mpg'] = np.nan
for car in cars_cleaned_df.iterrows():
    if car[1]['name'] in name_map:
        
#         print(name_map[car[1]['name']])
#         print(insurance_data[insurance_data['assembled_name'] == name_map[car[1]['name']]])
#         print(name_map[car[1]['name']])
        i = insurance_data[insurance_data['assembled_name'] == name_map[car[1]['name']]]['assembled_name'].index
        insurance_data.loc[i,'price'] = car[1]['price']
        insurance_data.loc[i,'city_mpg'] = car[1]['city_mpg']
        insurance_data.loc[i,'highway_mpg'] = car[1]['highway_mpg']
print(len(index_set))

Int64Index([849], dtype='int64')
Int64Index([155], dtype='int64')
Int64Index([203], dtype='int64')
Int64Index([238], dtype='int64')
Int64Index([336], dtype='int64')
Int64Index([344], dtype='int64')
Int64Index([405], dtype='int64')
Int64Index([418], dtype='int64')
Int64Index([471], dtype='int64')
Int64Index([472], dtype='int64')
Int64Index([472], dtype='int64')
Int64Index([480], dtype='int64')
Int64Index([501], dtype='int64')
Int64Index([547], dtype='int64')
Int64Index([619], dtype='int64')
Int64Index([807], dtype='int64')
Int64Index([839], dtype='int64')
Int64Index([856], dtype='int64')
Int64Index([462], dtype='int64')
Int64Index([1302], dtype='int64')
Int64Index([1302], dtype='int64')
Int64Index([38], dtype='int64')
Int64Index([49], dtype='int64')
Int64Index([54], dtype='int64')
Int64Index([72], dtype='int64')
Int64Index([114], dtype='int64')
Int64Index([129], dtype='int64')
Int64Index([101], dtype='int64')
Int64Index([104], dtype='int64')
Int64Index([101], dtype='int64')
Int64Index([

Int64Index([1117], dtype='int64')
Int64Index([1121], dtype='int64')
Int64Index([1122], dtype='int64')
Int64Index([1123], dtype='int64')
Int64Index([1124], dtype='int64')
Int64Index([1125], dtype='int64')
Int64Index([1125], dtype='int64')
Int64Index([1126], dtype='int64')
Int64Index([1129], dtype='int64')
Int64Index([1131], dtype='int64')
Int64Index([1132], dtype='int64')
Int64Index([1133], dtype='int64')
Int64Index([1134], dtype='int64')
Int64Index([1135], dtype='int64')
Int64Index([1138], dtype='int64')
Int64Index([1139], dtype='int64')
Int64Index([1140], dtype='int64')
Int64Index([1145], dtype='int64')
Int64Index([1146], dtype='int64')
Int64Index([1147], dtype='int64')
Int64Index([1148], dtype='int64')
Int64Index([1152], dtype='int64')
Int64Index([1154], dtype='int64')
Int64Index([1156], dtype='int64')
Int64Index([1157], dtype='int64')
Int64Index([1158], dtype='int64')
Int64Index([1154], dtype='int64')
Int64Index([1163], dtype='int64')
Int64Index([1164], dtype='int64')
Int64Index([11

Int64Index([873], dtype='int64')
Int64Index([877], dtype='int64')
Int64Index([878], dtype='int64')
Int64Index([885], dtype='int64')
Int64Index([886], dtype='int64')
Int64Index([887], dtype='int64')
Int64Index([902], dtype='int64')
Int64Index([942], dtype='int64')
Int64Index([923], dtype='int64')
Int64Index([931], dtype='int64')
Int64Index([943], dtype='int64')
Int64Index([944], dtype='int64')
Int64Index([950], dtype='int64')
Int64Index([950], dtype='int64')
Int64Index([975], dtype='int64')
Int64Index([975], dtype='int64')
Int64Index([977], dtype='int64')
Int64Index([1012], dtype='int64')
Int64Index([1059], dtype='int64')
Int64Index([1035], dtype='int64')
Int64Index([1036], dtype='int64')
Int64Index([1051], dtype='int64')
Int64Index([1061], dtype='int64')
Int64Index([1086], dtype='int64')
Int64Index([1087], dtype='int64')
Int64Index([1088], dtype='int64')
Int64Index([1094], dtype='int64')
Int64Index([1096], dtype='int64')
Int64Index([1098], dtype='int64')
Int64Index([1099], dtype='int64

In [291]:
insurance_data[insurance_data['highway_mpg'] > 0]

,Make,Model,Price:AK,Price:AL,Price:AZ,Price:AR,Price:CA,Price:CO,Price:CT,Price:DE,...,Price:VT,Price:VA,Price:WA,Price:WV,Price:WI,Price:WY,assembled_name,price,city_mpg,highway_mpg
20,Acura,RDX,1258.5,1072.0,1292.5,1562.0,1946.0,1723.5,1693.0,1820.0,...,1055.5,1184.0,1019.0,1399.0,1416.0,943.5,Acura RDX,40595.0,21.0,27.0
21,Acura,RDX A-Spec,1266.0,1124.0,1326.5,1560.5,1959.0,1728.0,1668.5,1826.0,...,1037.0,1189.5,1026.0,1404.5,1428.5,941.5,Acura RDX A-Spec,45497.5,22.0,27.0
23,Acura,RDX Tech,1290.5,1099.5,1316.5,1611.0,1996.5,1773.5,1735.0,1871.0,...,1091.0,1216.0,1040.0,1421.5,1461.5,970.5,Acura RDX Tech,42795.0,22.0,28.0
38,Alfa-Romeo,Giulia,1907.0,1599.0,1923.0,2483.0,2980.0,2510.0,2246.0,2491.0,...,1499.0,1630.0,1618.0,2094.0,2013.0,1380.0,Alfa-Romeo Giulia,45190.0,23.0,31.0
49,Alfa-Romeo,Stelvio,1720.0,1470.0,1793.0,2265.0,2481.0,2346.0,2035.0,2293.0,...,1308.0,1583.0,1312.0,1883.0,1939.0,1291.0,Alfa-Romeo Stelvio,45940.0,22.0,29.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1439,Volvo,XC40 T5 Momentum,1596.0,1401.0,2277.0,1701.0,2169.0,2927.0,2876.0,1907.0,...,1243.0,1810.0,1249.0,1547.0,2094.0,1012.0,Volvo XC40 T5 Momentum,43280.0,22.0,30.0
1442,Volvo,XC60 T5 Momentum,1281.5,1193.0,2095.0,1621.0,1973.5,2469.0,2806.5,1830.5,...,977.5,1752.0,1051.5,1396.5,1494.0,959.0,Volvo XC60 T5 Momentum,49970.0,20.0,27.0
1443,Volvo,XC60 T5 R-Design,1293.0,1206.5,1741.0,1588.5,2008.5,2124.0,2205.0,1801.5,...,1019.5,1481.5,1053.0,1414.0,1475.5,963.5,Volvo XC60 T5 R-Design,53755.0,20.0,27.0
1452,Volvo,XC90 T5 Momentum,1329.0,1309.0,1486.0,1630.5,2109.0,1894.0,1704.5,1888.5,...,1125.5,1239.5,1136.0,1470.0,1613.5,1016.0,Volvo XC90 T5 Momentum,52005.0,18.0,26.0


In [293]:
insurance_msrp_data = insurance_data[insurance_data['price'] > 0]
insurance_msrp_data

,Make,Model,Price:AK,Price:AL,Price:AZ,Price:AR,Price:CA,Price:CO,Price:CT,Price:DE,...,Price:VT,Price:VA,Price:WA,Price:WV,Price:WI,Price:WY,assembled_name,price,city_mpg,highway_mpg
20,Acura,RDX,1258.5,1072.0,1292.5,1562.0,1946.0,1723.5,1693.0,1820.0,...,1055.5,1184.0,1019.0,1399.0,1416.0,943.5,Acura RDX,40595.0,21.0,27.0
21,Acura,RDX A-Spec,1266.0,1124.0,1326.5,1560.5,1959.0,1728.0,1668.5,1826.0,...,1037.0,1189.5,1026.0,1404.5,1428.5,941.5,Acura RDX A-Spec,45497.5,22.0,27.0
23,Acura,RDX Tech,1290.5,1099.5,1316.5,1611.0,1996.5,1773.5,1735.0,1871.0,...,1091.0,1216.0,1040.0,1421.5,1461.5,970.5,Acura RDX Tech,42795.0,22.0,28.0
38,Alfa-Romeo,Giulia,1907.0,1599.0,1923.0,2483.0,2980.0,2510.0,2246.0,2491.0,...,1499.0,1630.0,1618.0,2094.0,2013.0,1380.0,Alfa-Romeo Giulia,45190.0,23.0,31.0
49,Alfa-Romeo,Stelvio,1720.0,1470.0,1793.0,2265.0,2481.0,2346.0,2035.0,2293.0,...,1308.0,1583.0,1312.0,1883.0,1939.0,1291.0,Alfa-Romeo Stelvio,45940.0,22.0,29.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1439,Volvo,XC40 T5 Momentum,1596.0,1401.0,2277.0,1701.0,2169.0,2927.0,2876.0,1907.0,...,1243.0,1810.0,1249.0,1547.0,2094.0,1012.0,Volvo XC40 T5 Momentum,43280.0,22.0,30.0
1442,Volvo,XC60 T5 Momentum,1281.5,1193.0,2095.0,1621.0,1973.5,2469.0,2806.5,1830.5,...,977.5,1752.0,1051.5,1396.5,1494.0,959.0,Volvo XC60 T5 Momentum,49970.0,20.0,27.0
1443,Volvo,XC60 T5 R-Design,1293.0,1206.5,1741.0,1588.5,2008.5,2124.0,2205.0,1801.5,...,1019.5,1481.5,1053.0,1414.0,1475.5,963.5,Volvo XC60 T5 R-Design,53755.0,20.0,27.0
1452,Volvo,XC90 T5 Momentum,1329.0,1309.0,1486.0,1630.5,2109.0,1894.0,1704.5,1888.5,...,1125.5,1239.5,1136.0,1470.0,1613.5,1016.0,Volvo XC90 T5 Momentum,52005.0,18.0,26.0


In [295]:
insurance_msrp_data.to_csv('insurance_msrp_data.csv')

In [296]:
insurance_data[insurance_data['Make'] == 'Tesla']

,Make,Model,Price:AK,Price:AL,Price:AZ,Price:AR,Price:CA,Price:CO,Price:CT,Price:DE,...,Price:VT,Price:VA,Price:WA,Price:WV,Price:WI,Price:WY,assembled_name,price,city_mpg,highway_mpg
1257,Tesla,Model S 75D,3236.0,2295.0,3185.0,4226.0,3878.0,4221.0,3125.0,3755.0,...,2117.0,2770.0,2149.0,2982.0,3187.0,2401.0,Tesla Model S 75D,NaN,NaN,NaN
1258,Tesla,Model X 75D,2896.0,1909.0,2759.0,3858.0,3351.0,3947.0,2955.0,3378.0,...,1858.0,2536.0,1768.0,3184.0,2977.0,2198.0,Tesla Model X 75D,NaN,NaN,NaN
